In [1]:
#import os
#import path
#import numpy as np
import pandas as pd
import fastparquet

In [2]:
#Obtener los labels
def labels(data, columna):
    label_list = []
    count = 0
    for chunk in data:
        #label_list.append(chunk)
        for i in chunk[columna].unique().tolist(): #Se hace en todo el dataset para verificar la totalidad de Label
            label_list.append(i)
        count += 1
    print(count, " iteraciones")
    return set(label_list)

In [3]:
#Cargar el dataset y crear los archivos TSV y Parquet

def data4label(data, label):
    c_list = []
    for chunk in data:  
        # Seleccionar los datos de acuerdo a Label
        c_filter = chunk[chunk['Label'] == label]
        # Añadir los datos a la lista de objetos
        c_list.append(c_filter)
    # Concatenar los datos filtrados en un DataFrame 
    df = pd.concat(c_list)
    # Regresar el df convertido a archivo TSV
    df_tsv = df.to_csv(label.replace(" ", "_")+'-data.tsv', sep = '\t')
    print("Dataset ", label,".tsv listo")
    #Hacer el parquet
    df_pq = parquet(df, ['Country Code', 'Label Net Total'], label)
    print("Dataset ", label,".parquet listo")
    return df_tsv, df_pq

In [4]:
def parquet(df, cols, lab):
    aux = df[cols]
    aux = aux.groupby('Country Code').sum()
    aux['Label'] = lab
    return aux.to_parquet(lab.replace(" ", "_")+'.parquet', engine='fastparquet', compression='UNCOMPRESSED')

### Se cargan los datos con un chunksize de 10**6 para no saturar la memoria

In [5]:
size = 10**6 #El dataset se carga con un chunksize = 10**6 para evitar saturar la memoria

In [6]:
#Solo se usó para saber cuales eran las diferentes columnas
cols = ['Country Code', 'Label'] 
for i in range(len(cols)):
    df_col = pd.read_csv('dig_fake.csv', usecols=cols, chunksize=size)
    locals()[str(cols[i]).replace(" ", "_") + "_values"] = labels(df_col, cols[i])
    
print(Country_Code_values)
print(Label_values)

33  iteraciones
33  iteraciones
{nan, 'DK', 'SK', 'CR', 'EC', 'PE', 'BR', 'AD', 'NG', 'SN', 'CY', 'FI', 'NI', 'BH', 'DO', 'DZ', 'MY', 'SV', 'MX', 'AL', 'CA', 'PS', 'KW', 'AZ', 'PL', 'ME', 'LB', 'UA', 'CZ', 'CL', 'MN', 'KZ', 'TN', 'LU', 'NO', 'MD', 'ES', 'PT', 'IE', 'CH', 'SD', 'AE', 'GT', 'RU', 'CN', 'SE', 'HU', 'OM', 'JO', 'UG', 'RO', 'GB', 'IS', 'SA', 'NL', 'AU', 'IN', 'LV', 'HR', 'IL', 'KY', 'CO', 'RS', 'XK', 'PA', 'SI', 'AT', 'MZ', 'TZ', 'KG', 'KH', 'NZ', 'BO', 'EE', 'MT', 'ZZ', 'KE', 'TW', 'LT', 'QA', 'MA', 'PY', 'KR', 'TH', 'BA', 'EG', 'US', 'HK', 'ZA', 'IT', 'NP', 'DE', 'GR', 'JP', 'UY', 'SG', 'BY', 'AR', 'PH', 'FR', 'BE', 'BG', 'TR', 'ID', 'VN', 'HN', 'TT', 'KN', 'PK'}
{'debonair account', 'redundant substance', 'billowy rake'}


## Hacer Archivo TSV y Parquet

In [6]:
#Las siguientes columas son con las que se pide trabajar
columnas = ['Customer', 'Title_Description', 'UPC', 'Product ID', 'Artist', 'Product Title', 'Country Code', 
            'Transaction Type', 'Quantity', 'Label Net Total', 'Label']
Label_values = list(Label_values)
for i in range(len(Label_values)):
    df_chunk = pd.read_csv('dig_fake.csv', usecols=columnas, chunksize=size)
    data4label(df_chunk, Label_values[i])

Dataset  redundant substance .tsv listo
Dataset  redundant substance .parquet listo
Dataset  debonair account .tsv listo
Dataset  debonair account .parquet listo
Dataset  billowy rake .tsv listo
Dataset  billowy rake .parquet listo


In [7]:
#Revisar los archivos Parquet
df1 = pd.read_parquet('debonair_account.parquet')
df1.head()

,Label Net Total,Label
Country Code,,
AD,13.678537,debonair account
AE,498.749024,debonair account
AL,34.968387,debonair account
AR,2977.314064,debonair account
AT,10198.778365,debonair account


In [ ]:
#df1['Label'] = 'debonair account'

In [9]:
df2 = pd.read_parquet('redundant_substance.parquet')

In [10]:
df3 = pd.read_parquet('billowy_rake.parquet')

In [12]:
#Concatenar los archivos parquet (solo por comodiad)
df_parquet = pd.concat([df1,df2, df3])
df_parquet

,Label Net Total,Label
Country Code,,
AD,13.678537,debonair account
AE,498.749024,debonair account
AL,34.968387,debonair account
AR,2977.314064,debonair account
AT,10198.778365,debonair account
...,...,...
UY,460.252163,billowy rake
VN,857.964145,billowy rake
XK,2.471951,billowy rake
